In [3]:
import os
import json
import torch
from tqdm import tqdm
from transformers import BartTokenizer
from transformers import BartForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [4]:
articles = []
tags = []

In [5]:
for cursor in tqdm(os.listdir("./normolized_aticles")):
    articles.append(cursor)

100%|███████████████████████████████████████████████████████████████████████| 24940/24940 [00:00<00:00, 4156802.77it/s]


In [6]:
for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for i in tmp:
        tags.append(i)

100%|███████████████████████████████████████████████████████████████████████████| 24940/24940 [01:40<00:00, 247.97it/s]


In [7]:
element_counts = {}
for element in tqdm(tags):
    count = tags.count(element)
    element_counts[element] = count 

100%|█████████████████████████████████████████████████████████████████████████| 150929/150929 [03:49<00:00, 657.47it/s]


In [8]:
from operator import itemgetter

sorted_counts = sorted(element_counts.items(), key=itemgetter(1), reverse=True)

sorted_counts = sorted_counts[:10]
for element, count in sorted_counts:
    print(element, count)

проблема 676
python 607
apple 537
искусственный интеллект 507
программирование 481
игры 469
россия 430
информационная безопасность 407
машинное обучение 385
санкции 381


In [9]:
x_train = []
y_train = []

In [10]:
tags_top10 = []
for element, _ in sorted_counts:
        tags_top10.append(element)
        
print(tags_top10)
tags_top10_digit = []
index = 0
for tag in tags_top10:
    tmp = [tag, index]
    index += 1
    tags_top10_digit.append(tmp)

print(tags_top10_digit)


['проблема', 'python', 'apple', 'искусственный интеллект', 'программирование', 'игры', 'россия', 'информационная безопасность', 'машинное обучение', 'санкции']
[['проблема', 0], ['python', 1], ['apple', 2], ['искусственный интеллект', 3], ['программирование', 4], ['игры', 5], ['россия', 6], ['информационная безопасность', 7], ['машинное обучение', 8], ['санкции', 9]]


In [11]:
for cursor in tqdm(articles):
    with open(f'normolized_aticles/{cursor}', encoding='utf-8') as asset:
        data = json.load(asset)
    tmp = data["tags"]
    for tag in tmp:
        if tag in tags_top10:
            text = data["content"]
            x_train.append(text)
            for i in range(len(tags_top10)):
                if tag == tags_top10[i]:
                    y_train.append(i)

100%|██████████████████████████████████████████████████████████████████████████| 24940/24940 [00:03<00:00, 7315.44it/s]


In [12]:
texts = x_train
labels = y_train

In [13]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')


In [14]:
max_length = 1000

# Преобразование текстов в последовательности токенов и паддинг
input_ids = []
attention_masks = []

for text in tqdm(texts):
    encoded = tokenizer.encode_plus(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids.append(encoded['input_ids'].squeeze())
    attention_masks.append(encoded['attention_mask'].squeeze())

# Преобразование в тензоры PyTorch
input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)

# Вывод размерности тензоров
print("Input IDs shape:", input_ids.shape)
print("Attention Masks shape:", attention_masks.shape)

100%|██████████████████████████████████████████████████████████████████████████████| 4880/4880 [01:06<00:00, 73.17it/s]

Input IDs shape: torch.Size([4880, 1000])
Attention Masks shape: torch.Size([4880, 1000])


In [15]:
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']
dataset = TensorDataset(input_ids, attention_mask, torch.tensor(labels))

In [16]:
# Создание модели и оптимизатора
model = BartForSequenceClassification.from_pretrained('facebook/bart-base', num_labels=10)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Цикл обучения
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda'
model.to(device)
train_dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    
    for batch in tqdm(train_dataloader):
        batch = [item.to(device) for item in batch]
        input_ids, attention_mask, labels = batch
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {average_loss:.4f}")

100%|██████████████████████████████████████████████████████████████████████████████| 2440/2440 [15:15<00:00,  2.66it/s]


Epoch 1/10 - Loss: 1.5813


100%|██████████████████████████████████████████████████████████████████████████████| 2440/2440 [15:03<00:00,  2.70it/s]


Epoch 2/10 - Loss: 1.1446


100%|██████████████████████████████████████████████████████████████████████████████| 2440/2440 [21:50<00:00,  1.86it/s]


...

Epoch 8/10 - Loss: 0.4642


100%|██████████████████████████████████████████████████████████████████████████████| 2440/2440 [21:17<00:00,  1.91it/s]


Epoch 9/10 - Loss: 0.4244


100%|██████████████████████████████████████████████████████████████████████████████| 2440/2440 [20:19<00:00,  2.00it/s]

Epoch 10/10 - Loss: 0.3983


In [19]:
# Сохранение модели:
try:
    torch.save(model.state_dict(), "model.pt")
    print("Saved")
except SystemError:
    pass

Saved
